## 파일 불러오기
학습 데이터, 테스트 데이터 로딩

In [1]:
import pandas as pd
import numpy as np
np.set_printoptions(threshold=np.inf) # print all numpy values
from sklearn.preprocessing import LabelEncoder, LabelBinarizer, OneHotEncoder, MultiLabelBinarizer

In [2]:
categorical = ['주야', '요일', '발생지시도', '발생지시군구', '사고유형_대분류', '사고유형_중분류', '법규위반', 
            '도로형태_대분류', '도로형태', '당사자종별_1당_대분류', '당사자종별_2당_대분류']
numerical = ['사상자수', '사망자수', '중상자수', '경상자수','부상신고자수']

x_train_num = pd.read_csv('./교통사망사고정보/Kor_Train_교통사망사고정보(12.1~17.6).csv',encoding='cp949', usecols=numerical)

x_train_cat = pd.read_csv('./교통사망사고정보/Kor_Train_교통사망사고정보(12.1~17.6).csv',encoding='cp949', usecols=categorical)


In [3]:
x_test_num = pd.read_csv('./test_kor.csv',encoding='cp949', usecols=numerical)

x_test_cat = pd.read_csv('./test_kor.csv',encoding='cp949', usecols=categorical)

## One Hot Encdoing

In [4]:
all_data = pd.concat((x_test_cat.dropna(), x_train_cat))
# for col in all_data.select_dtypes(include=[np.object]).columns:
#     print(col, all_data[col].unique())

In [5]:
for column in all_data.select_dtypes(include=[np.object]).columns:
    x_train_cat[column] = x_train_cat[column].astype('category', categories = all_data[column].unique())
    x_test_cat[column] = x_test_cat[column].astype('category', categories = all_data[column].unique())

/home/iron/.local/lib/python3.5/site-packages/ipykernel_launcher.py:2: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  
/home/iron/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
x_train_cat = pd.get_dummies(data=x_train_cat)
x_test_cat = pd.get_dummies(data=x_test_cat)

In [7]:
# One Encoding Shape 확인
print(x_train_cat.shape)
print(x_test_cat.shape)

(25037, 328)
(50, 328)


# Deep Learning

In [8]:
import keras
from keras.layers import Dense, Input, LSTM, concatenate, Dropout, Conv2D, MaxPool2D, Embedding, Reshape, Conv1D
from keras.models import Sequential, Model
from keras.utils import np_utils
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from keras import metrics
from keras import backend as K

Using TensorFlow backend.


In [9]:
# Train Data Shape
print(x_train_cat.shape)
print(x_train_num.shape)
# Test Data Shape
print(x_test_cat.shape)
print(x_test_num.shape)

(25037, 328)
(25037, 5)
(50, 328)
(50, 5)


In [10]:
# Numeric cases
Case1 = ['사망자수','사상자수','경상자수']
Case2 = ['사상자수', '중상자수', '부상신고자수']
Case3 = ['사상자수', '중상자수', '경상자수' ]
Case4 = ['사망자수', '사상자수', '중상자수' ]

# Categorical cases
Case5 = ['사고유형_대분류', '사고유형_중분류', '법규위반']
Case6 = ['도로형태_대분류', '도로형태', '당사자종별_1당_대분류']
Case7 = ['도로형태_대분류', '도로형태', '당사자종별_2당_대분류']
Case8 = ['도로형태_대분류', '도로형태', '당사자종별_1당_대분류', '당사자종별_2당_대분류']
Case11 = ['발생지시도', '발생지시군구']
Case12 = ['요일', '사고유형_대분류', '사고유형_중분류']
Case13 = ['요일', '사고유형_중분류', '법규위반', '도로형태_대분류']

# Mixed cases
Case9 = ['사망자수', '사상자수', '발생지시군구']
Case10 = ['중상자수', '경상자수', '발생지시군구']
Case14 = ['사망자수', '사상자수', '주야', '당사자종별_1당_대분류']
Case15 = ['사상자수', '중상자수', '주야', '도로형태']

Cases = [Case1, Case2, Case3, Case4, Case5, Case6, Case7, Case8, Case9, Case10,
         Case11, Case12, Case13, Case14, Case15]

In [11]:
# case ranges

case1_range = [0, 1]
case2_range = [2, 3]
case3_range = [4, 6]
case4_range = [7, 9]
case5_range = [10, 19]
case6_range = [20, 22]
case7_range = [23, 25]
case8_range = [26, 29]
case9_range = [30, 31]
case10_range = [32, 34]
case11_range = [35, 39]
case12_range = [40, 41]
case13_range = [42, 44]
case14_range = [45, 46]
case15_range = [47, 49]

test_ranges = [case1_range, case2_range, case3_range, case4_range, case5_range, case6_range,
             case7_range, case8_range, case9_range, case10_range, case11_range, case12_range,
             case13_range, case14_range, case15_range]

In [12]:
# train, set result in new array
for (idx, case), range_x in zip(enumerate(Cases), test_ranges):
    print(idx+1, case, range_x)

1 ['사망자수', '사상자수', '경상자수'] [0, 1]
2 ['사상자수', '중상자수', '부상신고자수'] [2, 3]
3 ['사상자수', '중상자수', '경상자수'] [4, 6]
4 ['사망자수', '사상자수', '중상자수'] [7, 9]
5 ['사고유형_대분류', '사고유형_중분류', '법규위반'] [10, 19]
6 ['도로형태_대분류', '도로형태', '당사자종별_1당_대분류'] [20, 22]
7 ['도로형태_대분류', '도로형태', '당사자종별_2당_대분류'] [23, 25]
8 ['도로형태_대분류', '도로형태', '당사자종별_1당_대분류', '당사자종별_2당_대분류'] [26, 29]
9 ['사망자수', '사상자수', '발생지시군구'] [30, 31]
10 ['중상자수', '경상자수', '발생지시군구'] [32, 34]
11 ['발생지시도', '발생지시군구'] [35, 39]
12 ['요일', '사고유형_대분류', '사고유형_중분류'] [40, 41]
13 ['요일', '사고유형_중분류', '법규위반', '도로형태_대분류'] [42, 44]
14 ['사망자수', '사상자수', '주야', '당사자종별_1당_대분류'] [45, 46]
15 ['사상자수', '중상자수', '주야', '도로형태'] [47, 49]


In [13]:
def save_result():
    '''
    test file에 있는 값을 result file에 저장
    '''
    test_file = pd.read_csv('./test_kor_1.csv', encoding='cp949', names= [chr(y) for y in range(ord('A'),ord('P')+1)])
    result_file = pd.read_csv('./result_kor.csv', encoding='cp949')
    
    cols = result_file['열'].values
    rows = result_file['행'].values
    vals = result_file['값'].astype('str').values
    
    rows-=1
    
    for i, (row, col) in enumerate(zip(rows, cols)):      
        vals[i] = test_file[col][row]

    rows += 1

    with open('./result_kor1.csv', 'wb') as f:
        for i in result_file.index:
            np.savetxt(f,  np.c_[rows,cols,vals],  delimiter=",", fmt='%s', encoding='cp949', header='행,열,값', comments='')
    print('Save Success')

In [14]:
# 수치형 데이터 Case 함수

def numeric_case(case, start, end):
    
    K.clear_session()
    case_copy=case.copy()
    
    # Case 확인
    print("Case:", case)
    
    # Train Data
    X = x_train_num.drop(columns=case)
    X = pd.concat([X, x_train_cat], axis=1).values
    
    # Test Data
    X_test = x_test_num.drop(columns=case)
    X_test = pd.concat([X_test, x_test_cat],axis=1).values
    
    # Label Data
    if '사상자수' in case:       
        case_copy.remove('사상자수')
        print('사상자제거:', case_copy) 
    Y = x_train_num[case_copy].values
    
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    
    # Model define

    num_input = Input(shape=( X.shape[1], 1), name='num_input')
    x = LSTM(512, return_sequences=True)(num_input)
    x = LSTM(512)(x)
    x = Dense((int(len(X[0] + len(Y[0])) * 2 /3)), activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense((int(len(X[0] + len(Y[0])) * 2 /3)), activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense((int(len(X[0] + len(Y[0])) * 2 /3)), activation='relu')(x)
    x = Dropout(0.3)(x)
    num_output = Dense(len(Y[0]), name='num_output')(x)
    
    
    # Model define
#     num_input = Input(shape=(len(X[0]),), name='num_input')
#     x = Dense((int(len(X[0] + len(Y[0])) * 2 /3)), activation='relu')(num_input)
#     x = Dropout(0.3)(x)
#     x = Dense((int(len(X[0] + len(Y[0])) * 2 /3)), activation='relu')(x)
#     x = Dropout(0.3)(x)
#     x = Dense((int(len(X[0] + len(Y[0])) * 2 /3)), activation='relu')(x)
#     x = Dropout(0.3)(x)
#     num_output = Dense(len(Y[0]), name='num_output')(x)

    model = Model(inputs=num_input, outputs=num_output)

    model.compile(optimizer='adam',
                  loss='mse',
                  metrics=['accuracy'])

    learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=25, verbose=1, factor=0.5, min_lr=0.00000001)

    callbacks = [
        learning_rate_reduction, # learning_rate를 점차 감소시킴으로서 최적값에 접근하는 방식
        EarlyStopping('val_loss', patience=15)# val_loss이 최적값에서 멀어지는 순간 epoch가 남아도 학습 중지
    ]
    
    history = model.fit(X, Y, epochs=50, batch_size=128, callbacks=callbacks, validation_split=0.2 )
    
    
    # make a prediction
    Y_test = model.predict(X_test[start:end+1])
    
    del model
    
    return Y_test

In [15]:
# 범주형 데이터 Case 함수

def categorical_case(case, start, end):
    
    K.clear_session()
    
    # Case 확인
    print("Case:", case)
    
    col_name = [] # ex. '사고유형_대분류_차대차', '사고유형_대분류_차대사람', '사고유형_대분류_차량단독'
    label_name = [] #  ex. '차대차', '차대사람', '차량단독
    
    # One Hot Encoding 후 Columns 이름과 Columns에 들어 있는 값 
    for col in case:
        label_name.extend(all_data[col].unique()) 
        for name in all_data[col].unique():
            col_name.append(col+'_'+name)

    # Train Data 
    X = x_train_cat.drop(columns=col_name)
    X = pd.concat([X, x_train_num], axis=1).values

    # Test Data
    X_test = x_test_cat.drop(columns=col_name)
    X_test = pd.concat([X_test, x_test_num],axis=1).values
    
    # Label Data
    Y = x_train_cat[col_name].values
    
    
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    
    
    cat_input = Input(shape=( X.shape[1], 1), name='cat_input')
    x = LSTM(512)(cat_input)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    cat_output = Dense(len(Y[0]), activation='sigmoid', name='cat_output')(x)
    
    # Model define
#     cat_input = Input(shape=(len(X[0]),), name='cat_input')
#     x = Dense(512, activation='relu')(cat_input)
#     x = Dropout(0.3)(x)
#     x = Dense(512, activation='relu')(x)
#     x = Dropout(0.3)(x)
#     x = Dense(512, activation='relu')(x)
#     x = Dropout(0.3)(x)
#     x = Dense(256, activation='relu')(x)
#     x = Dropout(0.3)(x)
#     cat_output = Dense(len(Y[0]), activation='sigmoid', name='cat_output')(x)

    model = Model(inputs=cat_input, outputs=cat_output)

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=25, verbose=1, factor=0.5, min_lr=0.00000001)

    callbacks = [
        learning_rate_reduction, # learning_rate를 점차 감소시킴으로서 최적값에 접근하는 방식
        EarlyStopping('val_loss', patience=15), # val_loss이 최적값에서 멀어지는 순간 epoch가 남아도 학습 중지
        ]

    history = model.fit(X, Y, epochs=50, batch_size=128, callbacks=callbacks,validation_split=0.2 )
    
    # make a prediction
    Y_test = model.predict(X_test[start:end+1])
    

    '''
    예시 출력:
        사고유형_대분류 : 차량단독
        사고유형_중분류 : 공작물충돌
        법규위반 : 안전운전 의무 불이행
    '''
    result = []
    for cat in Y_test: 
        x_list = list(cat)
        label_name_x = label_name.copy()
        temp = []
        for col in case:
#             print(col, ':', label_name_x[x_list.index(max(x_list[0:len(all_data[col].unique())]))] )
            temp.append(label_name_x[x_list.index(max(x_list[0:len(all_data[col].unique())]))])
            del x_list[:len(all_data[col].unique())]
            del label_name_x[:len(all_data[col].unique())]
        result.append(temp)
        
        
    del model
    
    return np.array(result)


In [16]:
# 믹스형 데이터 Case 함수

def mix_case(case, n, start, end):
    '''
    case: Case에 해당되는 컬럼이 담긴 배열
    n: 범주형 데이터 수
    start: 해당 Case 테스트의 시작 인덱스
    end: 해당 Case 테스트의 마지막 인덱스
    '''
    
    case_copy=case.copy()
    
    # categorical
    col_name = []
    label_name = []
    cat_name = case_copy[-n:]
    
    for col in case_copy[-n:]:
        label_name.extend(all_data[col].unique()) 
        for name in all_data[col].unique():
            col_name.append(col+'_'+name)
    
    Y_cat = x_train_cat[col_name].values
    X1 = x_train_cat.drop(columns=col_name)
    X_test1 = x_test_cat.drop(columns=col_name)
    
    # categorical columns 삭제
    del case_copy[-n:]
    
    # numerical
    X2 = x_train_num.drop(columns=case_copy)
    X_test2 = x_test_num.drop(columns=case_copy)
    
    if '사상자수' in case: 
        case_copy.remove('사상자수')
    Y_num = x_train_num[case_copy].values
    
    X = pd.concat([X1, X2], axis=1).values
    X_test = pd.concat([X_test1, X_test2],axis=1).values
    
    
    # Model define
    cat_input = Input(shape=(len(X[0]),), name='cat_input')
    x = Dense(1024, activation='relu')(cat_input)
    x = Dropout(0.3)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    
    if n == 1:
        cat_output = Dense(len(Y_cat[0]), activation='softmax', name='cat_output')(x)
    elif n == 2:
        cat_output = Dense(len(Y_cat[0]), activation='sigmoid', name='cat_output')(x)
        
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    num_output = Dense(len(Y_num[0]), name='num_output')(x)

    model = Model(inputs=cat_input, outputs=[cat_output, num_output])

    if n==1:
        model.compile(optimizer='adam',
                      loss={'cat_output': 'categorical_crossentropy', 'num_output': 'mse'},
                      metrics=['accuracy'])
    elif n==2:
        model.compile(optimizer='adam',
                      loss={'cat_output': 'binary_crossentropy', 'num_output': 'mse'},
                      metrics=['accuracy'])
    
    
    learning_rate_reduction = ReduceLROnPlateau(monitor='cat_output_acc', 
                                            patience=25, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00000001)

    callbacks = [
        learning_rate_reduction, # learning_rate를 점차 감소시킴으로서 최적값에 접근하는 방식
        EarlyStopping('val_loss', patience=20), # val_loss이 최적값에서 멀어지는 순간 epoch가 남아도 학습 중지
    ]

    history = model.fit(X, {'cat_output':Y_cat, 'num_output':Y_num}, epochs=50, batch_size=128, callbacks=callbacks,validation_split=0.2 )
    
    
    # make a prediction
    Y_test = model.predict(X_test[start:end+1])    
    
    result = []
    for cat, num in zip(Y_test[0], Y_test[1]):
        x_list = list(cat)
        label_name_x = label_name.copy()
        temp = []
        for col in cat_name:
            temp.append(label_name_x[x_list.index(max(x_list[0:len(all_data[col].unique())]))])
            
            # 출력한 Column과 데이터 삭제
            del x_list[:len(all_data[col].unique())]
            del label_name_x[:len(all_data[col].unique())]
        temp.extend(num)
        result.append(temp)
        
    
    return np.array(result)
    

In [17]:
# set result array to each cases
def setResult(arr, predict, case):
    result_arr = arr
    
    if case == [int(x) for x in range(0, 2)]:
        for row in range(len(predict)):
            result_arr[row, 2] = predict[row, 0] # 사망자수
            result_arr[row, 5] = predict[row, 1] # 사상자수            
            result_arr[row, 3] = result_arr[row, 2] + result_arr[row, 4:7].sum() # 사상자수 = 사망자수 + (중상+경상+부상신고자수)
            
    elif case == [int(x) for x in range(2, 4)]:
        for row in range(len(predict)):
            result_arr[row, 4] = predict[row, 0]
            result_arr[row, 6] = predict[row, 1]
            result_arr[row, 3] = result_arr[row, 2] + result_arr[row, 4:7].sum()
            
    elif case == [int(x) for x in range(4, 7)]:
        for row in range(len(predict)):
            result_arr[row, 4] = predict[row, 0]
            result_arr[row, 5] = predict[row, 1]
            result_arr[row, 3] = result_arr[row, 2] + result_arr[row, 4:7].sum()
            
    elif case == [int(x) for x in range(7, 10)]:
        for row in range(len(predict)):
            result_arr[row, 2] = predict[row, 0]
            result_arr[row, 4] = predict[row, 1]
            result_arr[row, 3] = result_arr[row, 2] + result_arr[row, 4:7].sum()
            
    elif case == [int(x) for x in range(10,20)]:
        for row in range(len(predict)):
            result_arr[row, 9] = predict[row, 0] 
            result_arr[row, 10] = predict[row, 1] 
            result_arr[row, 11] = predict[row, 2] 
            print(result_arr[row, 11], predict[row, 2])
            
    elif case == [int(x) for x in range(20,23)]:
        for row in range(len(predict)):
            result_arr[row, 12] = predict[row, 0] 
            result_arr[row, 13] = predict[row, 1] 
            result_arr[row, 14] = predict[row, 2] 
            
    elif case == [int(x) for x in range(23,26)]:
        for row in range(len(predict)):
            result_arr[row, 12] = predict[row, 0] 
            result_arr[row, 13] = predict[row, 1] 
            result_arr[row, 15] = predict[row, 2] 
    
    elif case == [int(x) for x in range(26,30)]:
        for row in range(len(predict)):
            result_arr[row, 12] = predict[row, 0] 
            result_arr[row, 13] = predict[row, 1] 
            result_arr[row, 14] = predict[row, 2] 
            result_arr[row, 15] = predict[row, 3] 
    
    elif case == [int(x) for x in range(35,40)]:
        for row in range(len(predict)):
            result_arr[row, 7] = predict[row, 0] 
            result_arr[row, 8] = predict[row, 1] 
        
    elif case == [int(x) for x in range(40,42)]:
        for row in range(len(predict)):
            result_arr[row, 1] = predict[row, 0] 
            result_arr[row, 9] = predict[row, 1] 
            result_arr[row, 10] = predict[row, 2] 
        
    elif case == [int(x) for x in range(42,45)]:
        for row in range(len(predict)):
            result_arr[row, 1] = predict[row, 0] 
            result_arr[row, 10] = predict[row, 1] 
            result_arr[row, 11] = predict[row, 2] 
            result_arr[row, 12] = predict[row, 3] 
    
    elif case == [int(x) for x in range(30,32)]:
        for row in range(len(predict)):
            result_arr[row, 2] = float(predict[row, 1])
            result_arr[row, 3] = result_arr[row, 2] + result_arr[row, 4:7].sum()
            result_arr[row, 8] = predict[row, 0]
            
    elif case == [int(x) for x in range(32,35)]:
        for row in range(len(predict)):
            result_arr[row, 4] = predict[row, 1]
            result_arr[row, 5] = predict[row, 2]
            result_arr[row, 8] = predict[row, 0]
            
    elif case == [int(x) for x in range(45,47)]:
        for row in range(len(predict)):
            result_arr[row, 0] = predict[row, 0]
            result_arr[row, 2] = float(predict[row, 2])
            result_arr[row, 14] = predict[row, 1]
            result_arr[row, 3] = result_arr[row, 2] + result_arr[row, 4:7].sum()
            
    elif case == [int(x) for x in range(47,50)]:
        for row in range(len(predict)):
            result_arr[row, 0] = predict[row, 0]
            result_arr[row, 4] = float(predict[row, 2])
            result_arr[row, 13] = predict[row, 1]
            result_arr[row, 3] = result_arr[row, 2] + result_arr[row, 4:7].sum()
    
    print(result_arr)
    return result_arr
    

In [18]:
x_test = pd.read_csv('./test_kor.csv',encoding='cp949')

# setResult(each case array, predict array, start to end in each case):

In [19]:
test_arr = np.array([x_test.columns.values])
print(test_arr)

[['주야' '요일' '사망자수' '사상자수' '중상자수' '경상자수' '부상신고자수' '발생지시도' '발생지시군구'
  '사고유형_대분류' '사고유형_중분류' '법규위반' '도로형태_대분류' '도로형태' '당사자종별_1당_대분류'
  '당사자종별_2당_대분류']]


In [ ]:
# train, set result in new array
for (idx, case), case_range in zip(enumerate(Cases), test_ranges):
    # for matching case number +1 to idx
    print()
    # numeric case
    if (idx+1) <= 4:      
#         print(idx+1, case, case_range)
        Case_prediction = numeric_case(case, case_range[0], case_range[1])
        answer = setResult(x_test.loc[case_range[0]:case_range[1]].values, Case_prediction, [int(x) for x in range(case_range[0], case_range[1]+1)])
        test_arr = np.append(test_arr, answer, axis=0)
        
    # categorical case
    elif ((idx+1) >= 5 and (idx+1) <= 8) or ((idx+1) >= 11 and (idx+1) <= 13):
#         print(idx+1, case, case_range)
        Case_prediction = categorical_case(case, case_range[0], case_range[1])
        answer = setResult(x_test.loc[case_range[0]:case_range[1]].values, Case_prediction, [int(x) for x in range(case_range[0], case_range[1]+1)])
        test_arr = np.append(test_arr, answer, axis=0)

    # mixed case
    elif (idx+1) == 9 or (idx+1) == 10 or (idx+1) >= 14:
#         print(idx+1, case, case_range)
        
        if (idx+1) == 9 or (idx+1) == 10:
            num_categories = 1  # number of categories for each case
            
        if (idx+1) == 14 or (idx+1) == 15:
            num_categories = 2
            
        Case_prediction = mix_case(case, num_categories, case_range[0], case_range[1])
        answer = setResult(x_test.loc[case_range[0]:case_range[1]].values, Case_prediction, [int(x) for x in range(case_range[0], case_range[1]+1)])
        test_arr = np.append(test_arr, answer, axis=0)


Case: ['사망자수', '사상자수', '경상자수']
사상자제거: ['사망자수', '경상자수']
Train on 20029 samples, validate on 5008 samples
Epoch 1/50
20029/20029 [==============================] - 217s 11ms/step - loss: 0.7952 - acc: 0.9559 - val_loss: 0.7041 - val_acc: 0.9551
Epoch 2/50
20029/20029 [==============================] - 211s 11ms/step - loss: 0.6936 - acc: 0.9570 - val_loss: 0.7049 - val_acc: 0.9551
Epoch 3/50
20029/20029 [==============================] - 214s 11ms/step - loss: 0.6910 - acc: 0.9570 - val_loss: 0.7046 - val_acc: 0.9551
Epoch 4/50
20029/20029 [==============================] - 212s 11ms/step - loss: 0.6890 - acc: 0.9570 - val_loss: 0.7046 - val_acc: 0.9551
Epoch 5/50
20029/20029 [==============================] - 211s 11ms/step - loss: 0.6881 - acc: 0.9570 - val_loss: 0.7046 - val_acc: 0.9551
Epoch 6/50
20029/20029 [==============================] - 211s 11ms/step - loss: 0.6821 - acc: 0.9570 - val_loss: 0.7004 - val_acc: 0.9551
Epoch 7/50
20029/20029 [==============================] - 215

In [ ]:
test_arr

In [ ]:
with open('./test_kor_1.csv','wb',) as f:
        np.savetxt(f,  test_arr,  delimiter=",", fmt='%s', encoding='cp949')

In [ ]:
save_result()